# Model as a Service


```bash
uv add fastapi --extra standard
uv add streamlit
```

In [ ]:
import pickle
import mlflow
from fastapi import FastAPI
from pydantic import BaseModel
from mlflow import MlflowClient
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv(override=True)  # Carga las variables del archivo .env

mlflow.set_tracking_uri("databricks")
client = MlflowClient()

EXPERIMENT_NAME = "/Users/<tu_correo>/nyc-taxi-experiment"

run_ = mlflow.search_runs(order_by=['metrics.rmse ASC'],
                          output_format="list",
                          experiment_names=EXPERIMENT_NAME
                          )[0]

run_id = run_.info.run_id

run_uri = f"runs:/{run_id}/preprocessor"

client.download_artifacts(
    run_id=run_id,
    path='preprocessor',
    dst_path='.'
)

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_model = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

In [ ]:
def preprocess(input_data):

    input_dict = {
        'PU_DO': input_data.PULocationID + "_" + input_data.DOLocationID,
        'trip_distance': input_data.trip_distance,
    }
    X = dv.transform([input_dict])

    # Names depend on sklearn version
    try:
        cols = dv.get_feature_names_out()
    except AttributeError:
        cols = dv.get_feature_names()

    # 
    X_df = pd.DataFrame(X.toarray(), columns=cols)

    return X_df

def predict(input_data):

    X_val = preprocess(input_data)

    return champion_model.predict(X_val)

In [ ]:
app = FastAPI()

class InputData(BaseModel):
    PULocationID: str
    DOLocationID: str
    trip_distance: float


@app.post("api/v1/predict")
def predict_endpoint(input_data: InputData):
    result = predict(input_data)[0]
    return {"prediction": float(result)}

# Streamlit


In [ ]:
import streamlit as st
import requests
import json

st.write("""
# Application to predict the time for the NYC taxi trips
""")

st.sidebar.header('User Input Parameters')

def user_input_features():
    PU = st.sidebar.text_input("PU Location ID", "80")
    DO = st.sidebar.text_input("DO Location ID", "60")
    trip_distance = st.sidebar.number_input("Trip Distance", value=10, min_value=1, max_value=100)

    input_dict = {
        'PULocationID': PU,
        'DOLocationID': DO,
        'trip_distance': trip_distance,
    }

    return input_dict


input_dict = user_input_features()

### Para llamar una API desde streamlit

In [ ]:
if st.button('Predict'):
    response = requests.post(
        url="http://127.0.0.1:8000/predict",
        data=json.dumps(input_dict)
    )

    st.write(f"El tiempo estimado del viaje es: {response.json()['prediction']} minutos")